<a href="https://colab.research.google.com/github/rzyunanda/Machine_Learning/blob/main/Pgd_on_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt



# Definisi Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

# Load Data
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# Inisialisasi Model dan Optimizer
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Fungsi untuk melakukan PGD attack
def pgd_attack(model, images, labels, eps, alpha, iters):
    original_images = images.clone().detach()  # Save the original images
    images = images.clone().detach().requires_grad_(True)

    for _ in range(iters):
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()

        with torch.no_grad():
            # Add perturbations and project back to the epsilon ball
            images += images.grad.sign() * alpha
            images = torch.clamp(images, min=original_images - eps, max=original_images + eps)
            images = torch.clamp(images, min=0, max=1)  # Ensure pixel values are valid

        # Detach and reattach requires_grad to reset gradients correctly
        images = images.detach().requires_grad_(True)

        plt.show()

    return images


# Training model
model.train()
for data, target in train_loader:
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    # Menerapkan PGD Attack pada data training
    perturbed_data = pgd_attack(model, data, target, eps=0.3, alpha=0.01, iters=40)
    output = model(perturbed_data)
    loss = criterion(output, target)
    print(loss.item())

# Simpan model jika diperlukan
torch.save(model.state_dict(), 'mnist_model.pth')


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.